# **PART 1**

In [21]:
%%shell
gdown --quiet 1-IeoZDwT5wQzBUpsaS5B6vTaP-2ZBkam
pip --quiet install pyspark

In [4]:
COMPLAINTS_FN = 'complaints_sample.csv'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

In [5]:
dt = sc.textFile(COMPLAINTS_FN, use_unicode=True).cache()
dt.take(2)

['Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID',
 '2015-12-31,Bank account or service,Checking account,"Making/receiving payments, sending money",,,,FIRSTBANK PUERTO RICO,PR,00902,Older American,N/A,Referral,2016-02-04,Closed with explanation,Yes,No,1723943']

In [7]:
def combine(part_id,rows):
    if part_id == 0:
        next(rows)
    reserve = ''
    for row in rows:
        if row[:4].isnumeric()==True:
            if len(reserve)>0:
                yield reserve
            reserve = row
        else:
            reserve+=row

In [8]:
import re
def modify(row):
    pattern = r'"[^"]*"'
    matches = re.finditer(pattern,row)
    positions = []
    for match in matches:
        positions.append((match.start(), match.end()))
    quot_modified = []
    if len(positions)>0:
        for x in positions:
            target_quot = row[x[0]:x[1]]
            replacement_quot = re.sub(",","_", target_quot) 
            quot_modified.append((replacement_quot,x))
        output_row = row
        for y in quot_modified:
            output_row = output_row[:y[1][0]]+y[0]+output_row[y[1][1]:]
        return output_row.lower()
    else:
        return row.lower()

In [23]:
task1 = dt.mapPartitionsWithIndex(combine)\
        .map(modify)\
        .map(lambda x: x.split(','))\
        .map(lambda x: ((x[0][:4],x[1],x[7]),1))
task1.take(5)

[(('2015', 'bank account or service', 'firstbank puerto rico'), 1),
 (('2016', 'bank account or service', 'firstbank puerto rico'), 1),
 (('2016', 'bank account or service', 'wells fargo & company'), 1),
 (('2017', 'checking or savings account', 'comerica'), 1),
 (('2018', 'checking or savings account', 'wells fargo & company'), 1)]

In [25]:
check = task1.filter(lambda x: x[0][0] in ['2018'])

In [26]:
def remove_quot(row):
    if row[0][0].isalpha():
        return ','.join(row)
    else:
        out = [row[0][1:-1],row[1],row[2],row[3],row[4]]
        return ','.join(out)
outputTask1 = task1.groupByKey()\
        .mapValues(lambda x: sum(x))\
        .map(lambda x: ((x[0][0],x[0][1]),(x[1],1)))\
        .groupByKey()\
        .mapValues(lambda x: (sum([a[0] for a in x]),max([a[0] for a in x]),sum([a[1] for a in x])))\
        .map(lambda x: [x[0][1],x[0][0],str(x[1][0]),str(x[1][2]),str(round(100*x[1][1]/x[1][0],0))[:-2]])\
        .map(remove_quot)\
        .sortBy(lambda x: x[:])
outputTask1.take(5)

['bank account or service,2015,1,1,100',
 'bank account or service,2016,2,2,50',
 'checking or savings account,2017,1,1,100',
 'checking or savings account,2018,20,10,25',
 'checking or savings account,2019,461,72,13']

# part 2

In [13]:
!pip install google-cloud-dataproc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.5/307.5 kB 6.6 MB/s eta 0:00:00


In [14]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Zc6HKz6pAGkzlVoU9kJyvglTIwlyV5&prompt=consent&access_type=offline&code_challenge=jX7U30EPvDU99QRIY6vfeW801ze7HaPwbOQZpEZgkFY&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VPqzUzJNiUFKOqkSxJlvdD4iV4U-dURF-ROj03DeUix7D7fU3Sw3rddVQsaoZ0Z1w

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [15]:
!gcloud projects list

PROJECT_ID            NAME              PROJECT_NUMBER
bda-12345             bda-12345         369021837527
directed-will-384217  My First Project  913257743120


In [16]:
!gcloud config set project bda-12345
!gcloud config set compute/region us-west1
!gcloud config set compute/zone us-west1-a
!gcloud config set dataproc/region us-west1

Updated property [core/project].
Updated property [compute/region].
Updated property [compute/zone].
Updated property [dataproc/region].


In [17]:
!gcloud dataproc clusters create bdm-hw3 --enable-component-gateway --region us-west1 --zone us-west1-a --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 500 --image-version 2.0-debian10 --project bda-12345

Waiting on operation [projects/bda-12345/regions/us-west1/operations/15c45f85-43ab-37b9-b9be-031d676cd33d].

Created [https://dataproc.googleapis.com/v1/projects/bda-12345/regions/us-west1/clusters/bdm-hw3] Cluster placed in zone [us-west1-a].


In [18]:
!gcloud dataproc clusters list

NAME     PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE        SCHEDULED_DELETE
bdm-hw3  GCE       2                                       RUNNING  us-west1-a


In [19]:
%%writefile BDM_HW3_23499013_Jin.py
#!/usr/bin/python
COMPLAINTS = 'gs://bdma/data/complaints.csv'
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
dt = sc.textFile(COMPLAINTS, use_unicode=True).cache()

def combine(part_id,rows):
    if part_id == 0:
        next(rows)
    reserve = ''
    for row in rows:
        if row[:4].isnumeric()==True:
            if len(reserve)>0:
                yield reserve
            reserve = row
        else:
            reserve+=row

import re
def modify(row):
    pattern = r'"[^"]*"'
    matches = re.finditer(pattern,row)
    positions = []
    for match in matches:
        positions.append((match.start(), match.end()))
    # change all the commas-',' in the quotation mark to _, so that I can split with comma later on.
    # and use the modified sentence in the quotation mark to replace the original sentence in the quotation mark.
    quot_modified = []
    if len(positions)>0:
        for x in positions:
            target_quot = row[x[0]:x[1]]
            replacement_quot = re.sub(",","_", target_quot) 
            quot_modified.append((replacement_quot,x))
        output_row = row
        for y in quot_modified:
            output_row = output_row[:y[1][0]]+y[0]+output_row[y[1][1]:]
        return output_row.lower()
    else:
        return row.lower()

task1 = dt.mapPartitionsWithIndex(combine)\
        .map(modify)\
        .map(lambda x: x.split(','))\
        .map(lambda x: ((x[0][:4],x[1],x[7]),1))

def remove_quot(row):
    if row==none:
      pass
    elif row[0][0].isalpha():
        return ','.join(row)
    else:
        out = [row[0][1:-1],row[1],row[2],row[3],row[4]]
        return ','.join(out)
        
outputTask1 = task1.groupByKey()\
        .mapValues(lambda x: sum(x))\
        .map(lambda x: ((x[0][0],x[0][1]),(x[1],1)))\
        .groupByKey()\
        .mapValues(lambda x: (sum([a[0] for a in x]),max([a[0] for a in x]),sum([a[1] for a in x])))\
        .map(lambda x: [x[0][1],x[0][0],str(x[1][0]),str(x[1][2]),str(round(100*x[1][1]/x[1][0],0))[:-2]])\
        .map(remove_quot)\
        .sortBy(lambda x: x)
outputTask1.saveAsTextFile('gs://bdma/shared/2023_spring/HW3/23499013_Jin/report.csv')

Writing BDM_HW3_23499013_Jin.py


In [20]:
!gcloud dataproc jobs submit pyspark --cluster bdm-hw3 BDM_HW3_23499013_Jin.py -- gs://bdma/data/complaints.csv gs://bdma/shared/2023_spring/HW3/23499013_Jin

In [20]:
!gsutil ls gs://bdma/shared/2023_spring/HW3/